# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,65.86,51,0,5.30,US,1697551226
1,1,prince rupert,54.3161,-130.3201,53.69,66,100,13.80,CA,1697551226
2,2,hawaiian paradise park,19.5933,-154.9731,70.03,95,1,1.36,US,1697551227
3,3,grytviken,-54.2811,-36.5092,36.12,88,100,8.99,GS,1697551010
4,4,college,64.8569,-147.8028,36.90,78,100,1.01,US,1697551228


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
# Configure the map plot
import warnings
warnings.filterwarnings('ignore')
map_plot = city_data_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    hover_cols=["City", "Country", "Humidity"],
    xlabel="Longitude",
    ylabel="Latitude",
    alpha=0.5,
)

# Show the map plot
display(map_plot)

:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
# Filter the DataFrame based on your ideal weather conditions
ideal_weather_conditions = (
    (city_data_df["Max Temp"] > 10) & (city_data_df["Max Temp"] < 30) &
    (city_data_df["Wind Speed"] < 8) 
)

# Create a copy of the filtered DataFrame
hotel_df = city_data_df.loc[ideal_weather_conditions].copy()

# Drop any rows with null values
hotel_df.dropna(inplace=True)

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,anadyr,64.7500,177.4833,23.50,93,40,6.71,RU,1697551232
22,22,iqaluit,63.7506,-68.5145,23.70,68,97,3.00,CA,1697551240
131,131,tommot,58.9564,126.2925,27.73,98,100,2.93,RU,1697551313
135,135,aykhal,66.0000,111.5000,18.07,98,84,5.01,RU,1697551315
158,158,khandyga,62.6667,135.6000,13.93,88,100,2.35,RU,1697551328
167,167,amga,60.8953,131.9608,25.99,52,100,7.00,RU,1697551333
226,226,sisimiut,66.9395,-53.6735,19.04,79,37,6.08,GL,1697551372
408,408,namtsy,62.7161,129.6658,25.03,49,100,7.31,RU,1697551486


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = hotel_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,anadyr,64.7500,177.4833,23.50,93,40,6.71,RU,1697551232,
22,22,iqaluit,63.7506,-68.5145,23.70,68,97,3.00,CA,1697551240,
131,131,tommot,58.9564,126.2925,27.73,98,100,2.93,RU,1697551313,
135,135,aykhal,66.0000,111.5000,18.07,98,84,5.01,RU,1697551315,
158,158,khandyga,62.6667,135.6000,13.93,88,100,2.35,RU,1697551328,
167,167,amga,60.8953,131.9608,25.99,52,100,7.00,RU,1697551333,
226,226,sisimiut,66.9395,-53.6735,19.04,79,37,6.08,GL,1697551372,
408,408,namtsy,62.7161,129.6658,25.03,49,100,7.31,RU,1697551486,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {
    "categories":categories,
    "limit":1,
    "apiKey":geoapify_key}
    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
  
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
     
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
anadyr - nearest hotel: Гостевой дом
iqaluit - nearest hotel: Frobisher Inn
tommot - nearest hotel: Томмот
aykhal - nearest hotel: Уют
khandyga - nearest hotel: Президентская
amga - nearest hotel: "АммА"
sisimiut - nearest hotel: Sømandshjemmet
namtsy - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,anadyr,64.7500,177.4833,23.50,93,40,6.71,RU,1697551232,Гостевой дом
22,22,iqaluit,63.7506,-68.5145,23.70,68,97,3.00,CA,1697551240,Frobisher Inn
131,131,tommot,58.9564,126.2925,27.73,98,100,2.93,RU,1697551313,Томмот
135,135,aykhal,66.0000,111.5000,18.07,98,84,5.01,RU,1697551315,Уют
158,158,khandyga,62.6667,135.6000,13.93,88,100,2.35,RU,1697551328,Президентская
167,167,amga,60.8953,131.9608,25.99,52,100,7.00,RU,1697551333,"""АммА"""
226,226,sisimiut,66.9395,-53.6735,19.04,79,37,6.08,GL,1697551372,Sømandshjemmet
408,408,namtsy,62.7161,129.6658,25.03,49,100,7.31,RU,1697551486,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [80]:
# Configure the map plot
import warnings
warnings.filterwarnings('ignore')
map_plot = city_data_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"],
    xlabel="Longitude",
    ylabel="Latitude",
    alpha=0.5,
)

# Show the map plot
display(map_plot)

:Scatter   [Lng]   (Lat,Humidity,City,Country)